# Quick for beginners

In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

## Build model

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

In [7]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[-0.13447364,  0.05210891, -0.12825075, -0.42678085,  0.18400076,
         0.34131718,  0.43266496,  1.1057056 , -0.28769174, -1.1458851 ]],
      dtype=float32)

In [19]:
tf.nn.softmax(predictions).numpy()

array([[0.07471989, 0.09004675, 0.07518633, 0.05578133, 0.10274198,
        0.12024572, 0.13174722, 0.25824937, 0.06410538, 0.02717602]],
      dtype=float32)

In [20]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [21]:
loss_fn(y_train[:1], predictions).numpy()

2.118218

In [22]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [23]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 40us/sample - loss: 0.2982 - accuracy: 0.9130
Epoch 2/5
60000/60000 [==============================] - 2s 30us/sample - loss: 0.1441 - accuracy: 0.9574
Epoch 3/5
60000/60000 [==============================] - 2s 30us/sample - loss: 0.1078 - accuracy: 0.9676
Epoch 4/5
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0866 - accuracy: 0.9739
Epoch 5/5
60000/60000 [==============================] - 2s 28us/sample - loss: 0.0763 - accuracy: 0.9764


In [12]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0717 - accuracy: 0.9771


[0.07167452388429082, 0.9771]

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.60279540e-09, 1.03314790e-09, 7.38316021e-06, 9.03531152e-04,
        6.30885405e-12, 1.49436929e-07, 2.79333627e-14, 9.99087930e-01,
        3.69343581e-07, 6.45025068e-07],
       [8.39165352e-07, 6.81101950e-03, 9.93151426e-01, 3.26570553e-05,
        2.71884999e-13, 1.15787816e-06, 6.45626926e-07, 2.57752958e-10,
        2.25472127e-06, 2.28281456e-11],
       [6.41668692e-08, 9.99309897e-01, 9.32936600e-05, 7.19145601e-06,
        4.71298772e-05, 3.54589076e-07, 9.96188760e-07, 2.40724927e-04,
        2.97265506e-04, 3.03044135e-06],
       [9.99899149e-01, 1.73790182e-09, 7.73185457e-05, 1.92352161e-08,
        2.85603193e-07, 4.25539810e-07, 1.37829875e-05, 4.69298675e-06,
        3.18130020e-07, 4.06024083e-06],
       [1.49068455e-07, 1.72286449e-10, 8.78304036e-07, 1.86274178e-08,
        9.94307935e-01, 5.92564675e-07, 1.27388989e-06, 2.79038204e-06,
        7.15639828e-07, 5.68566844e-03]], dtype=float32)>

# Quick for experts

In [67]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [68]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [69]:
x_train.shape

(60000, 28, 28, 1)

In [60]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [61]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation="relu")
        self.flatten = Flatten()
        self.d1 = Dense(128, activation="relu")
        self.d2 = Dense(10)
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
model = MyModel()

In [62]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [63]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [64]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [65]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [66]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.13659557700157166, Accuracy: 95.95000457763672, Test Loss: 0.07496730238199234, Test Accuracy: 97.43999481201172
Epoch 2, Loss: 0.0431833490729332, Accuracy: 98.66999816894531, Test Loss: 0.05606967583298683, Test Accuracy: 98.18999481201172
Epoch 3, Loss: 0.022525927051901817, Accuracy: 99.23666381835938, Test Loss: 0.05131297931075096, Test Accuracy: 98.43000030517578
Epoch 4, Loss: 0.012901896610856056, Accuracy: 99.57666015625, Test Loss: 0.06402900815010071, Test Accuracy: 98.27999